In [1]:
# 1번 문제
# 02-2.ipynb 파일을 수정하여 다른 정보를 추출하여 저장하는 코드를 작성하세요. 
# 무게를 추출해도 되고, 크기를 추출해도 되고, 아니면 카테고리 정보나 저자 정보를 추출해도 됩니다.
# 주어진 코드와 같이 원하는 새로운 정보를 포함하는 신규 데이터 프레임을 만드는 것까지 수행하세요.
# ☞ 무게를 추출하여 신규 데이터 프레임을 생성하겠습니다.

In [15]:
import gdown
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
gdown.download('https://bit.ly/3q9SZix', '20s_best_book.json', quiet=False)

Downloading...
From: https://bit.ly/3q9SZix
To: C:\Users\LG\20s_best_book.json
100%|██████████████████████████████████████████████████████████████████████████████| 92.9k/92.9k [00:00<00:00, 448kB/s]


'20s_best_book.json'

In [25]:
books_df = pd.read_json('20s_best_book.json')
books = books_df[['no','ranking','bookname','authors','publisher','publication_year','isbn13']]

In [32]:
def get_weight_info(isbn):
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        r = requests.get(url.format(isbn), headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        prd_info = soup.find('a', attrs={'class': 'gd_name'})
        if prd_info is None:
            return ''
        
        detail_url = 'http://www.yes24.com' + prd_info['href']
        r = requests.get(detail_url, headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        prd_detail = soup.find('div', attrs={'id': 'infoset_specific'})
        if prd_detail is None:
            return ''
        
        prd_tr_list = prd_detail.find_all('tr')
        for tr in prd_tr_list:
            th = tr.find('th')
            td = tr.find('td')
            if th and th.get_text(strip=True) == '쪽수, 무게, 크기':
                info_parts = [part.strip() for part in td.get_text(strip=True).split('|')]
                if len(info_parts) >= 2:
                    return info_parts[1]  # 무게는 두 번째 항목 (예: '496g')
        return ''
    except Exception as e:
        print(f"[예외] {isbn}: {e}")
        return ''

In [33]:
get_weight_info('9791190090018')

'496g'

In [34]:
weight_list = []
for i, isbn in enumerate(books['isbn13']):
    print(f"[{i+1}] ISBN {isbn} 처리 중...")
    weight = get_weight_info(isbn)
    weight_list.append(weight)
    time.sleep(1)

[1] ISBN 9791190090018 처리 중...
[2] ISBN 9791165341909 처리 중...
[3] ISBN 9791188862290 처리 중...
[4] ISBN 9788954672214 처리 중...
[5] ISBN 9788936434267 처리 중...
[6] ISBN 9788936434243 처리 중...
[7] ISBN 9791165300005 처리 중...
[8] ISBN 9788972756194 처리 중...
[9] ISBN 9788936477196 처리 중...
[10] ISBN 9788954641630 처리 중...
[11] ISBN 9791190090261 처리 중...
[12] ISBN 9788936438036 처리 중...
[13] ISBN 9791164136452 처리 중...
[14] ISBN 9788934972464 처리 중...
[15] ISBN 9788954651820 처리 중...
[16] ISBN 9788936434120 처리 중...
[17] ISBN 9791190786355 처리 중...
[18] ISBN 9788936437534 처리 중...
[19] ISBN 9788998441074 처리 중...
[20] ISBN 9788936433598 처리 중...
[21] ISBN 9788934985068 처리 중...
[22] ISBN 9791130621968 처리 중...
[23] ISBN 9788954646079 처리 중...
[24] ISBN 9791196067694 처리 중...
[25] ISBN 9788937473098 처리 중...
[26] ISBN 9788996991342 처리 중...
[27] ISBN 9791189015381 처리 중...
[28] ISBN 9788954655972 처리 중...
[29] ISBN 9788998274795 처리 중...
[30] ISBN 9791191071030 처리 중...
[31] ISBN 9791195522125 처리 중...
[32] ISBN 9791160

In [36]:
books = books_df[['no','ranking','bookname','authors','publisher','publication_year','isbn13']].copy()

In [37]:
books['weight_info'] = weight_list

In [38]:
books.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13,weight_info
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,496g
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,358g
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,296g
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,412g
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,388g


In [39]:
#  1번을 성공했으면 2번은 옵션입니다.
# 이번에는 쪽수와 함께 위에서 추출한 다른 정보를 동시에 저장하여 처리하는 코드를 작성해보세요.
# 즉, def get_page_cnt(isbn): 함수 내에 쪽수와 다른 정보를 함께 추출해냅니다. 이후 이를 활용하여 두 개의 컬럼을 추가하는 코드를 작성해보세요.

In [40]:
def get_page_and_weight(isbn):
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        r = requests.get(url.format(isbn), headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, 'html.parser')

        prd_info = soup.find('a', attrs={'class': 'gd_name'})
        if prd_info is None:
            return None, None

        detail_url = 'http://www.yes24.com' + prd_info['href']
        r = requests.get(detail_url, headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, 'html.parser')

        prd_detail = soup.find('div', attrs={'id': 'infoset_specific'})
        if prd_detail is None:
            return None, None

        prd_tr_list = prd_detail.find_all('tr')
        for tr in prd_tr_list:
            th = tr.find('th')
            td = tr.find('td')
            if th and th.get_text(strip=True) == '쪽수, 무게, 크기':
                info_parts = [part.strip() for part in td.get_text(strip=True).split('|')]
                # 쪽수: 숫자만 추출
                page_cnt = int(''.join(filter(str.isdigit, info_parts[0]))) if len(info_parts) >= 1 else None
                weight_info = info_parts[1] if len(info_parts) >= 2 else None
                return page_cnt, weight_info
        return None, None
    except Exception as e:
        print(f"[예외] {isbn}: {e}")
        return None, None


In [41]:
page_list = []
weight_list = []

In [42]:
for i, isbn in enumerate(books['isbn13']):
    print(f"[{i+1}] ISBN {isbn} 처리 중...")
    page, weight = get_page_and_weight(isbn)
    page_list.append(page)
    weight_list.append(weight)
    time.sleep(1)

[1] ISBN 9791190090018 처리 중...
[2] ISBN 9791165341909 처리 중...
[3] ISBN 9791188862290 처리 중...
[4] ISBN 9788954672214 처리 중...
[5] ISBN 9788936434267 처리 중...
[6] ISBN 9788936434243 처리 중...
[7] ISBN 9791165300005 처리 중...
[8] ISBN 9788972756194 처리 중...
[9] ISBN 9788936477196 처리 중...
[10] ISBN 9788954641630 처리 중...
[11] ISBN 9791190090261 처리 중...
[12] ISBN 9788936438036 처리 중...
[13] ISBN 9791164136452 처리 중...
[14] ISBN 9788934972464 처리 중...
[15] ISBN 9788954651820 처리 중...
[16] ISBN 9788936434120 처리 중...
[17] ISBN 9791190786355 처리 중...
[18] ISBN 9788936437534 처리 중...
[19] ISBN 9788998441074 처리 중...
[20] ISBN 9788936433598 처리 중...
[21] ISBN 9788934985068 처리 중...
[22] ISBN 9791130621968 처리 중...
[23] ISBN 9788954646079 처리 중...
[24] ISBN 9791196067694 처리 중...
[25] ISBN 9788937473098 처리 중...
[26] ISBN 9788996991342 처리 중...
[27] ISBN 9791189015381 처리 중...
[28] ISBN 9788954655972 처리 중...
[29] ISBN 9788998274795 처리 중...
[30] ISBN 9791191071030 처리 중...
[31] ISBN 9791195522125 처리 중...
[32] ISBN 9791160

In [43]:
books['page_cnt'] = page_list
books['weight_info'] = weight_list

In [44]:
books.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13,weight_info,page_cnt
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,496g,344.0
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,358g,300.0
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,296g,228.0
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,412g,340.0
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,388g,264.0
